In [2]:
import xarray as xr
from pandas import to_datetime
from dask.diagnostics import ProgressBar
from dask import persist
import time
from distributed import Client,LocalCluster


In [3]:
cluster = LocalCluster()          # Fully-featured local Dask cluster
client = cluster.get_client()

2024-09-27 20:25:07,774 - tornado.application - ERROR - Uncaught exception GET /status/ws (127.0.0.1)
HTTPServerRequest(protocol='http', host='127.0.0.1:8787', method='GET', uri='/status/ws', version='HTTP/1.1', remote_ip='127.0.0.1')
Traceback (most recent call last):
  File "c:\Users\ls2236\AppData\Local\anaconda3\envs\arco\Lib\site-packages\tornado\websocket.py", line 938, in _accept_connection
    open_result = handler.open(*handler.open_args, **handler.open_kwargs)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ls2236\AppData\Local\anaconda3\envs\arco\Lib\site-packages\tornado\web.py", line 3301, in wrapper
    return method(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ls2236\AppData\Local\anaconda3\envs\arco\Lib\site-packages\bokeh\server\views\ws.py", line 149, in open
    raise ProtocolError("Token is expired.")
bokeh.protocol.exceptions.ProtocolError: Token is expired.


In [4]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 6
Total threads: 24,Total memory: 63.25 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:53511,Workers: 6
Dashboard: http://127.0.0.1:8787/status,Total threads: 24
Started: Just now,Total memory: 63.25 GiB
Comm: tcp://127.0.0.1:53549,Total threads: 4
Dashboard: http://127.0.0.1:53551/status,Memory: 10.54 GiB
Nanny: tcp://127.0.0.1:53514,


In [6]:
date_i = '1998-01-01'
date_f = '1998-12-31'


latN = 90
latS = 20
lonW = -150
lonE = -40

Var = '2m_temperature'

slice_xy = True
slice_t = True

In [7]:
def roll_longitude(ds):
    return ds.assign_coords(longitude=(((ds.longitude + 180) % 360) - 180)).sortby('longitude')

def slice_temporal(ds,datei,datef):
    return ds.sel(time=slice(date_i,date_f))

def slice_spatial(ds,latN,latS,lonW,lonE):
    return ds.sel(latitude = slice(latN,latS),longitude=slice(lonW,lonE))

In [8]:
reanalysis = xr.open_zarr(
    'gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3',
    chunks={"time": 48},
    storage_options=dict(token='anon'),
)

In [9]:
pre_data = reanalysis

if slice_t:
    pre_data = slice_temporal(pre_data,date_i,date_f)

pre_data = roll_longitude(pre_data)

if slice_xy:
    pre_data = slice_spatial(pre_data,latN,latS,lonW,lonE)

data = pre_data[Var]

In [21]:
data

<xarray.DataArray '2m_temperature' (time: 8760, latitude: 281, longitude: 441)> Size: 4GB
dask.array<getitem, shape=(8760, 281, 441), dtype=float32, chunksize=(48, 281, 441), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float32 1kB 90.0 89.75 89.5 89.25 ... 20.5 20.25 20.0
  * time       (time) datetime64[ns] 70kB 1998-01-01 ... 1998-12-31T23:00:00
  * longitude  (longitude) float32 2kB -150.0 -149.8 -149.5 ... -40.25 -40.0
Attributes:
    long_name:   2 metre temperature
    short_name:  t2m
    units:       K

In [18]:
tmin = data.groupby("time.dayofyear").min(dim='time')
tmax = data.groupby("time.dayofyear").max(dim='time')
#daily_data['2m_temperature_min'] = daily_data["2m_temperature"]

In [20]:
#daily_data.rename({'2m_temperature' : '2m_temperature_min'})
daily_data

<xarray.DataArray '2m_temperature' (dayofyear: 365, latitude: 281,
                                    longitude: 441)> Size: 181MB
dask.array<stack, shape=(365, 281, 441), dtype=float32, chunksize=(1, 281, 441), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float32 1kB 90.0 89.75 89.5 89.25 ... 20.5 20.25 20.0
  * longitude  (longitude) float32 2kB -150.0 -149.8 -149.5 ... -40.25 -40.0
  * dayofyear  (dayofyear) int64 3kB 1 2 3 4 5 6 7 ... 360 361 362 363 364 365
Attributes:
    long_name:   2 metre temperature
    short_name:  t2m
    units:       K

In [18]:
data.groupby("time.dayofyear")

<DataArrayGroupBy, grouped over 1 grouper(s), 365 groups in total:
    'dayofyear': 365 groups with labels 1, 2, 3, 4, 5, ..., 362, 363, 364, 365>

In [12]:
#output filename
time_max=to_datetime(data.time.max().values).strftime("%Y%m%d")
time_min=to_datetime(data.time.min().values).strftime("%Y%m%d")
print(time_min,time_max)

19980101 19990101


In [13]:
file_name = "ERA5/arco-era5/data/{}_{}_{}.nc".format(Var,time_min,time_max)
print(file_name)

ERA5/arco-era5/data/2m_temperature_19980101_19990101.nc


In [14]:
lazy = data.to_netcdf(file_name,compute=False)

In [ ]:
%%time
with ProgressBar():
    #data.to_netcdf(file_name)
    lazy.compute()